In [1]:
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn import model_selection
from sklearn import svm
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.metrics import confusion_matrix
import sklearn.datasets as datasets
from sklearn.metrics import classification_report as report
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score as accuarcy
from numpy import argmax

In [2]:
df = pd.read_csv("data.csv",names=["Chemical_formula","A","B","In_literature","Valence_A","Valence_B","Radius_A","Radius_B","Lowest_distortion","Formation_energy","Stability","Magnetic_moment","Volume_per_atom","Band_gap","a","b","c","alpha","beta","gamma","Vacancy_energy"])

In [3]:
df.drop(df.index[(df["Valence_B"] == "5")],axis=0,inplace=True)
df.drop(df.index[(df["Valence_A"] == "5")],axis=0,inplace=True)
df.drop(df.index[(df["Valence_B"] == "4")],axis=0,inplace=True)
df.drop(df.index[(df["Valence_A"] == "4")],axis=0,inplace=True)
df.drop(df.index[(df["Valence_A"] == "element not in BV")],axis=0,inplace=True)
df.drop(df.index[(df["Valence_B"] == "element not in BV")],axis=0,inplace=True)
df.drop(df.index[(df["Vacancy_energy"] =="-")],axis=0,inplace=True)
df['Valence_A'] = df['Valence_A'].replace(['not balanced'], '0') 
df['Valence_A'] = df['Valence_A'].replace(['3'], '1') 
df['Valence_B'] = df['Valence_B'].replace(['not balanced'], '0') 
df['Valence_B'] = df['Valence_B'].replace(['3'], '1') 
df['Lowest_distortion'] = df['Lowest_distortion'].replace(['cubic'], '0') 
df['Lowest_distortion'] = df['Lowest_distortion'].replace(['orthorhombic'], '1')
df['Lowest_distortion'] = df['Lowest_distortion'].replace(['rhombohedral'], '2')
df['Lowest_distortion'] = df['Lowest_distortion'].replace(['tetragonal'], '3')
df['Magnetic_moment'] = df['Magnetic_moment'].replace(['-'], '0.00')
df['Vacancy_energy'] = df['Vacancy_energy'].replace(['-'], '0.00')
df['In_literature'] = df['In_literature'].replace(["FALSE"], '0') 
df['In_literature'] = df['In_literature'].replace(["TRUE"], '1')

In [4]:
df[1:]

,Chemical_formula,A,B,In_literature,Valence_A,Valence_B,Radius_A,Radius_B,Lowest_distortion,Formation_energy,...,Magnetic_moment,Volume_per_atom,Band_gap,a,b,c,alpha,beta,gamma,Vacancy_energy
75,Ag2O3,Ag,Ag,0,0,0,1.28,0.95,0,0.351,...,0.00,12.997,0,4.02,4.02,4.02,90,90,90,-2.694
76,AgAlO3,Ag,Al,0,0,0,1.28,0.54,1,-1.51,...,0.00,9.898,0,5.094,5.323,7.3,90,90,90,-1.341
79,AgBO3,Ag,B,0,1,1,0.75,0.27,0,-0.35,...,0.00,8.138,0,3.439,3.439,3.439,90,90,90,-2.641
80,AgBaO3,Ag,Ba,0,0,0,1.28,1.35,1,-0.989,...,0.00,15.631,0.247,6.636,6.739,6.99,90,90,90,-6.975
81,AgBeO3,Ag,Be,0,0,0,1.28,0.45,0,-0.616,...,0.00,8.781,0,3.528,3.528,3.528,90,90,90,-3.058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5322,ZrTmO3,Zr,Tm,0,0,0,0.89,0.96,0,-2.315,...,0.2,14.496,0,4.169,4.169,4.169,90,90,90,-3.645
5326,ZrYO3,Zr,Y,0,0,0,0.89,0.9,0,-2.126,...,0.2,15.277,0,4.243,4.243,4.243,90,90,90,-4.92
5327,ZrYbO3,Zr,Yb,0,0,0,0.89,0.95,1,-3.455,...,0,13.136,4.007,5.558,5.726,8.254,90,90,90,-6.177
5328,ZrZnO3,Zr,Zn,0,0,0,0.89,0.74,0,-1.63,...,0.001,10.804,0,3.78,3.78,3.78,90,90,90,-0.762


In [5]:
features=["Formation_energy","Stability","Magnetic_moment","Volume_per_atom","Band_gap","Vacancy_energy"]
#,"Radius_A","Radius_B","Valence_A","Valence_B"
#Best possible combination if we want to remove radius A and radius B
X = df.loc[1:, features].values
Y = df.loc[1:, "Lowest_distortion"].values

In [6]:
X

array([['0.351', '0.576', '0.00', '12.997', '0', '-2.694'],
       ['-1.51', '0.235', '0.00', '9.898', '0', '-1.341'],
       ['-0.35', '1.074', '0.00', '8.138', '0', '-2.641'],
       ...,
       ['-3.455', '0.205', '0', '13.136', '4.007', '-6.177'],
       ['-1.63', '1.21', '0.001', '10.804', '0', '-0.762'],
       ['-2.382', '0.906', '0.382', '13.915', '0', '3.218']], dtype=object)

In [7]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split (X, Y, test_size=0.2, random_state=25)

In [8]:
counter = Counter(Y_train)
for k,v in counter.items():
    per = v / len(Y_train) * 100
    print('Class=%s, n=%s (%.3f%%)' % (k, v, per))

Class=1, n=678 (32.117%)
Class=2, n=101 (4.784%)
Class=0, n=1293 (61.251%)
Class=3, n=39 (1.847%)


In [9]:
oversample = SMOTE()
X_train, Y_train = oversample.fit_resample(X_train, Y_train)

In [10]:
counter = Counter(Y_train)
for k,v in counter.items():
    per = v / len(Y_train) * 100
    print('Class=%s, n=%s (%.3f%%)' % (k, v, per))

Class=1, n=1293 (25.000%)
Class=2, n=1293 (25.000%)
Class=0, n=1293 (25.000%)
Class=3, n=1293 (25.000%)


**SUPPORT VECTOR MACHINE-LINEAR KERNEL**

In [11]:
#without weights
SVM_linear = svm.SVC(kernel='linear', C=100,gamma=0.01,random_state=31).fit(X_train, Y_train)
print(SVM_linear.score(X_test, Y_test))

0.8068181818181818


In [12]:
prediction_Y=SVM_linear.predict(X_test)

In [13]:
cr = report(Y_test, prediction_Y)
cm = confusion_matrix(Y_test, prediction_Y)
print(cr)
print(cm)

              precision    recall  f1-score   support

           0       0.99      0.97      0.98       321
           1       0.87      0.56      0.68       168
           2       0.19      0.44      0.27        25
           3       0.19      0.64      0.30        14

    accuracy                           0.81       528
   macro avg       0.56      0.65      0.56       528
weighted avg       0.89      0.81      0.83       528

[[312   3   2   4]
 [  2  94  44  28]
 [  1   7  11   6]
 [  0   4   1   9]]


In [14]:
#setting weights so that points which exist in literature has more weight
weights=[]
for i in df["In_literature"]:
    if i=='0':
        weights=np.append(weights,np.array([1]))
    if i=='1':
        weights=np.append(weights,np.array([5]))
    else:
        pass
weights.shape

(2639,)

In [ ]:
#NOT WORKING
#with weights

SVM_linear2 = svm.SVC(kernel='linear',C=100,gamma=0.01)
SVM_linear2.fit(X_train, Y_train, sample_weight=np.ones(len(X_train)))
    #want to make it so that its not np.ones but rater weights, which consideres literature value
    #, which is not over sampled giving different shapes for feqtures
print(SVM_linear2.score(X_test,Y_test))

In [ ]:
prediction_Y=SVM_linear2.predict(X_test)
cr = report(Y_test, prediction_Y)
cm = confusion_matrix(Y_test, prediction_Y)
print(cr)
print(cm)

**SUPPORT VECTOR MACHINE-GAUSSIAN RBF KERNEL**

In [17]:
#without weights
SVM_rbf = svm.SVC(kernel='rbf', C=100,random_state=31).fit(X_train, Y_train)
print(SVM_rbf.score(X_test, Y_test))

0.8768939393939394


In [18]:
prediction_Y=SVM_rbf.predict(X_test)
cr = report(Y_test, prediction_Y)
cm = confusion_matrix(Y_test, prediction_Y)
print(cr)
print(cm)

              precision    recall  f1-score   support

           0       0.98      0.97      0.98       321
           1       0.90      0.77      0.83       168
           2       0.32      0.48      0.39        25
           3       0.31      0.64      0.42        14

    accuracy                           0.88       528
   macro avg       0.63      0.72      0.65       528
weighted avg       0.91      0.88      0.89       528

[[312   3   3   3]
 [  3 130  21  14]
 [  1   9  12   3]
 [  1   3   1   9]]


**SUPPORT VECTOR MACHINE-SIGMOID KERNEL**

In [19]:
#without weights
SVM_sigmoid = svm.SVC(kernel='sigmoid', C=100,random_state=31).fit(X_train, Y_train)
print(SVM_sigmoid.score(X_test, Y_test))

0.42992424242424243


In [20]:
prediction_Y=SVM_sigmoid.predict(X_test)
cr = report(Y_test, prediction_Y)
cm = confusion_matrix(Y_test, prediction_Y)
print(cr)
print(cm)

              precision    recall  f1-score   support

           0       0.65      0.53      0.58       321
           1       0.31      0.33      0.32       168
           2       0.03      0.04      0.03        25
           3       0.02      0.07      0.03        14

    accuracy                           0.43       528
   macro avg       0.25      0.24      0.24       528
weighted avg       0.50      0.43      0.46       528

[[169 113  24  15]
 [ 68  56  15  29]
 [ 12  10   1   2]
 [ 11   2   0   1]]


**SUPPORT VECTOR MACHINE-POLYNOMIAL KERNEL**

In [21]:
#without weights
SVM_poly = svm.SVC(kernel ='poly', degree = 3, C=100,random_state=31).fit(X_train, Y_train)
print(SVM_sigmoid.score(X_test, Y_test))

0.42992424242424243


In [22]:
prediction_Y=SVM_poly.predict(X_test)
cr = report(Y_test, prediction_Y)
cm = confusion_matrix(Y_test, prediction_Y)
print(cr)
print(cm)

              precision    recall  f1-score   support

           0       0.99      0.96      0.98       321
           1       0.86      0.66      0.75       168
           2       0.22      0.32      0.26        25
           3       0.20      0.71      0.31        14

    accuracy                           0.83       528
   macro avg       0.57      0.66      0.57       528
weighted avg       0.89      0.83      0.85       528

[[309   3   2   7]
 [  2 111  26  29]
 [  1  12   8   4]
 [  0   3   1  10]]
